# Importação das bibliotecas necessárias

In [1]:
import sys
import os
sys.path.append('/home/jovyan/work')

from utils.AlinharETL import AlinharETL
from pyspark.sql import types as T

# Crie uma instância da classe AlinharETL

In [2]:
# Crie uma instância da classe
bucket = "bronze"
datamart = "dadosabertosreceita"
extrator_bronze = AlinharETL(bucket,datamart)

2024-10-15 07:35:46,186 - INFO - Iniciando Sessão Spark.


# Parâmetros para leitura do arquivo

In [3]:
schema = T.StructType([
    T.StructField("codigo", T.StringType(), True),
    T.StructField("descricao", T.StringType(), True),
])

In [4]:
ano_mes = extrator_bronze.encontrar_ano_mes("dadosabertosreceita")
local = f"s3a://landing/dadosabertosreceita/{ano_mes}/paises/*.csv"

In [5]:
config_adicional = {
    'sep': ';',
    'encoding': 'latin1'
}

# Leitura do arquivo

In [6]:
df_compliance_receita_paises  = extrator_bronze.criar_view_temporaria_arquivo(local, 'compliance_paises',schema,**config_adicional)

2024-10-15 07:35:51,250 - INFO - Lista de yaml carregada com sucesso
2024-10-15 07:35:56,130 - INFO - Criando view temporaria.


# Gravação no datalake

In [7]:
extrator_bronze.caminho_tabela_delta = 's3a://bronze/dadosabertosreceita/paises'

In [8]:
extrator_bronze.salvar_delta(df_compliance_receita_paises, 'overwrite')

2024-10-15 07:35:56,144 - INFO - Aguarde... Persistindo dados (overwrite)
2024-10-15 07:36:14,426 - INFO - Dados persistidos com sucesso
2024-10-15 07:36:14,428 - INFO - s3a://bronze/dadosabertosreceita/paises
2024-10-15 07:36:14,429 - INFO - Aguarde... Realizando optimize
2024-10-15 07:36:25,974 - INFO - Optimize executado com sucesso.
2024-10-15 07:36:25,976 - INFO - Aguarde... Realizando vacuum
2024-10-15 07:36:49,635 - INFO - Vacuum executado com sucesso.


# Encerra sessão spark

In [9]:
extrator_bronze.parar_sessao()

2024-10-15 07:36:50,025 - INFO - Sessão Spark finalizada.
